$\color{red}{\text{Make sure you run each of these blocks in order, or things won't work correctly!}}$

In [ ]:
!git clone https://github.com/irawoodring/shared.git
from IPython.display import Audio, display

Cloning into 'shared'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 7 (delta 0), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), 1.29 MiB | 7.49 MiB/s, done.


# Ugh, pointers.

Most anyone who has programmed in C before will tell you that pointers suck.  The truth though, is that they are a powerful, flexible, and simple mechanism for accessing memory.  The very idea is intuitive; every byte in a computer's memory needs an address, and a pointer is a variable that holds one of those addresses.  Just like every house needs an address so the post office can find it - it just makes sense.

The reason people usually struggle with pointers, is that C is a language that struggles with **context**.  This means that to determine the meaning of some C constructs, you need to examine how they are used.  C was written in a time when many computers didn't have a large character set - and some characters needed to be reused for multiple purposes.  In fact, your professor worked with computers that didn't have lower-case letters in their character sets in the early '80s!  This is one reason why in C, you might see the "*" used in a variety of ways:

- ```/* I don't know what this code does. */``` - In a comment
- ```a * 3``` - In multiplication
- ```*a = 42``` - Dereferencing a pointer
- ```int* a``` - Declaring a pointer

etc.  So it really isn't that *pointers* are hard, it is more like the syntax and context of C can be tough.  But that's ok; you just have to learn the rules.

Let's go over some definitions.

## Definitions

You probably already know that computers use their memory to store data, and that when referring to memory in a program we often use **variables**.  Like many languages (not all), C creates some variables **statically** - this means at compile time they are built into the compiled code for a program.  C can also create variables **dynamically** - meaning while the program is running.  Variables that are created statically are managed by C.  Dynamically created variables you must manage yourself.

C requires us to **declare** a variable before we use it.  In C, we declare a variable by giving its type and name:

```C
int x;    // Create enough space to store an integer (usually 32 bits)
float y;  // Create enough space for a float (usually 32 bits)
double z; // Create enough space for a double (usually 64 bits)
char t;   // Create enough space for a char (usually 8 bits)
char* s;  // Create enough space for an address to a variable (usually 64 bits)
```

I say "usually" in the comments above because it depends on the actual system.  C doesn't dictate the size of a type (though many people think it does).  Instead, it dictates the **minimum size** of a type.  For instance, most people will say that a byte is 8 bits (and it is on most systems).  But the truth is, the C standard says it must be a minimum of 8 bits.  There are some machines that have 24 bit bytes (though they are rare)!  Before C was standardized there were even machines with 7 bit bytes.

Just like many other languages, C lets us **define** variables as well.  Defining a variable means giving it a value.  Sometimes we declare and define a variable all at once.

```C
int x = 42;   // Declaration and definition
y = 3.14;     // Definition only
s = "Hello world.!" // Definition only
s = &t;       // Definition only
```

## Ok.  Tell me more about variables.

You may think you already undrestand variables, but you may not fully.  Many people will think of a variable as a memory location that holds a value.  This is only partially true.  A variable is really a sextuple (a tuple with six entries) of the following values:

- **A name** - this is what us fleshbags will call this memory location.
- **An address** - this is where in memory the variable starts.
- **A type** - this is how we interpret the bits starting at a location, and how many bits we use for that type of value.
- **A value** - the actual bits we store.
- **A scope** - the parts of the code that are allowed to access this value.
- **A lifetime** - how long the variable will stay in memory.

>**Sidenote:** A lot of times people will get scope and lifetime confused.  Scope is the **where** in our program the variable can be accessedd.  Lifetime is the **when** it is created.

Imagine we have a program with the following code:

```C
int function1(){
  int x = 21 * 2;
  return x;
}

void function2(float y){
  y = y * 2;
}

int main(int argc, char** argv){
  int x = function1();
}
```

The C runtime keeps track of variables in something called a **symbol table**.  You may think of it something like this:

| Name        | Address            | Value              | Type   | Scope | Lifetime |
|-------------|--------------------|--------------------|--------|-------|----------|
| function1.x | 0x0000000100000eb0 | 42                 | 32 bit int    | l     |          |
| function2.y | 0x0000000100000ccc | 3.14               | 32 bit float  | l     |          |
| main.x      | 0x000000010000001a | 1701               | 32 bit int    | g     |          |
| main.argc   | 0x000000010000000f | 0                  | 32 bit int   | l     |          |
| main.argv   | 0x0000000100000010 | 0x0000000100000000 | 8 bit char** | l     |          |

Keep in mind that this is a trivial example and is a bit different from an actual symbol table.  There are other things besides variables that you will find in a symbol table as well.  For instance - functions!  After all, the code for a function has to go in memory somewhere too.  Also, values will change while the program runs.  I've included lifetime here, but it isn't always specifically listed.  Lifetime can be a measure of how long the variable entry is in the symbol table.

# Ok.  I get the idea.  How do I use this?

So, hopefully you understand variables better now.  In C we cannot use a variable unless it is declared at compile time.  If we want to declare memory at run-time, we **have to declare a variable that will hold the address of that memory at compile time**.  Consider the following use case:

> A programmer wants to keep a list of numbers in memory.  She will read those numbers off of a disk, out of a file.  She doesn't know how many numbers are in the file, but she wants to read them all.  How do we handle this in C?

The crux of this problem is really knowing how much memory we need to reserve at compile time.  **We do not know, and will not know how much memory we need for the numbers until we open the file and inspect it - at run time.**. Even if we knew at one point how many numbers were in the file, it could change after our program is compiled, as people edit the file.  So how much memory do we reserve?  We could reserve an arbitrarily large amount - say a million bytes.  But this might not be enough.  Or, it is far too much, and we've wasted a ton of space.

But, we don't actually need to know how many integers are in the file at compile time.  When our program starts to run, we can inspect the file and determine how many bytes we need.  Operating systems provide functions to do this.  Then, we can use the ```malloc``` system-call to ask for that many bytes.  This causes C to ask the operating system for a chunk of memory of the size we need.  **What need to keep track in our progam of where the operating system chooses to store those bytes.** The "where" for a variable is just a memory location.  How do we store a memory location?  You guessed it - that's what a pointer is for!

## Pointers

A pointer is just a variable that holds a memory address.  Nothing more, nothing less.  Sometimes I lose people here, but it isn't that complicated.  We have variables that for ```int```s, ```float```s, ```double```s, ```char```s, and all kinds of other types.  People tend to get confused because of context.  When we declare an ```int```, we type

```
int x;
```

When we declare a pointer to an ```int```, we type one of the following:

```
int* x;
int * x;
int *x;
```

It doesn't matter where the space goes, it all means the same thing.  But, we need to keep in mind that an ```int``` is as different from an ```int*``` as a cougar is from a watermelon.  one holds an integer value.  The other, holds the address of a variable that holds an integer value.  They have two very different purposes.

Back in the day, all pointers were of type ```void*```.  As compilers grew more powerful though, they gained the ability to ensure they were pointing to the correct type of information.  So these days, we give them a type.  But the fact is, **all pointers hold an address to some memory location on your system.** An address is just a number that represents a memory location on the machine.  And variables are great for holding numbers!

## An example

So let's go back to our symbol table and see what it might look like to solve our problem of reading integers off a disk and storing them.  We might have code like the following:

```C
int main(int arc, char** argv){
  int fs = get_file_size("my_integer_filename");
  int* data = malloc(fs);
  read_ints_from_file(data, "my_integer_filename");
}
```

I've got two made-up functions in this code - ```get_file_size()``` and ```read_ints_from_file()O```, but I've named them so you can guess what they do.  Our symbol table might look something like this:

| Name        | Address            | Value              | Type   | Scope | Lifetime |
|-------------|--------------------|--------------------|--------|-------|----------|
| main.fs | 0x000000010000f3a9 | 100                 | 32 bit int    | l     |          |
| main.data | 0x000000010000abba |     | 64 bit int pointer    | l     |          |

Before our program runs, we can see that we have space reserved for a 32-bit ```int``` and a 64 bit ```int*```.  These 96 bits will be reserved within our program's compiled code.  You may notice that something is missing though.  We have enough space for **an address for where our numbers will be stored.**. We don't actually have enough storage for the numbers.

At run-time, our symbol table will be loaded into memory.  When the program calls the ```malloc``` (memory allocation function), we pass in the number of bytes we need (```fs``` in this code).  This causes the operating system to find an area of memory big enough to store that many bytes, and to return to us the address where that memory starts.  So once our program performs the malloc, our symbol table might look more like this (notice this is before the ```read_ints_from_file()``` call):

| Name        | Address            | Value              | Type   | Scope | Lifetime |
|-------------|--------------------|--------------------|--------|-------|----------|
| main.fs | 0x000000010000f3a9 | 100                 | 32 bit int    | l     |          |
| main.data | 0x000000010000abba |   0x000000020000001a  | 64 bit int pointer    | l     |          |
|  | 0x000000020000001a |     | 3200 bits dynamically allocated for ints    | l     |          |

Why doesn't the 3rd entry have a name?  Well, the truth is computers don't need names at all.  Computers refer to memory with addresses.  Names are for us.  In this case, we don't need to name the address where we will store those values, as our code already has a variable to hold the address.  If we need to access the address, we will just ask that variable how to find it.  Notice how the value for the second entry is the address of the 3rd?  That's all a pointer is - an address.

# Some actual code

Let's go back to reading a file from a disk.  We don't know how big the file is at compile time; even if we did, it might change before we run our program.  So we need to figure out how big it is once our program starts running.  Let's imagine that we had the following file:

In [ ]:
%%writefile my_textfile.txt
Hello world!

Writing my_textfile.txt


Let's see how big this file is:

In [ ]:
!ls -l my_textfile.txt

-rw-r--r-- 1 root root 13 Sep 16 00:44 my_textfile.txt


It looks like the file takes up 13 bytes.  If you count the characters you will see there are only 12. On the system we are using, a newline character is automatically added to a textfile.  That character puts us up to 13.  If you modify the block above and re-run it, you will likely get a different number.

So, if we want to read this file into memory, we need an area in memory that is at least the number of bytes of the file.  I want to write a function that I can call that will load *any* file into memory for me, regardless of size.  I just want the function to take a filename and a pointer variable, and to load the file into memory and set the variable to the address where that data starts.  Since I won't know how many bytes the file is until I run this function, I want the function to return the number of bytes it reads.  The signature of such a function might look like ```read_data``` below:

In [ ]:
%%writefile file_tools.h

#include <stdlib.h>

/**********************************************************************
 * Read a file at the path, filename.  Put the address of where
 * the data is stored inside of the "data" variable.
 **********************************************************************/

size_t read_data(char* filename, unsigned char** data);



/**********************************************************************
 * Write a file.  Given a filename, an amount of data, and an address,
 *  write that many bytes to disk.
 **********************************************************************/

size_t write_data(char* filename, size_t data_size, unsigned char* data);





Writing file_tools.h


The actual code for the function might look like this:

In [ ]:
%%writefile file_tools.c

#include "file_tools.h"
#include <stdio.h>

size_t read_data(char* filename, unsigned char** data){
  // When we open a file, we keep that location
  // as a file pointer, so declare one.
  FILE* fp;

  // Open the file for reading.
  fp = fopen(filename, "r");

  // Go to the end of the file.
  fseek(fp, 0L, SEEK_END);

  // ftell says where we are in the file.  Since we went to the
  // end, this will tell us how many bytes are in the file.
  size_t file_size = ftell(fp);

  // Go back to the beginning.
  fseek(fp, 0L, SEEK_SET);

  // Now, we know how many bytes we need.  Ask for that much space,
  // and store the address.
  *data = malloc(file_size);

  // Read the data and store it in our location.
  fread(*data, 1, file_size, fp);

  // Close the file.
  fclose(fp);

  // Return the number of bytes we found.
  return file_size;
}

size_t write_data(char* filename, size_t data_size, unsigned char* data){
  //Open the file for writing in binary mode
  FILE* fp = fopen(filename, "wb");

  //size_t fwrite(const void *ptr, size_t size, size_t nmemb,FILE *stream);
  //On  success, fwrite() return the number of items written. (man fwrite)
  size_t bytes_written = fwrite(data, 1, data_size, fp);

  //Checks to make sure all of the data was read
  if (bytes_written != data_size) {
        printf("Error writing all data to %s\n", filename);
        fclose(fp);
        return 0;
  }

  //Every file that is opened using fopen() must be closed
  fclose(fp);

  return bytes_written;
}

Writing file_tools.c


And here is how we might use that function:

In [ ]:
%%writefile reversal.c

#include <stdbool.h>
#include <stdio.h>
#include <string.h>
#include "file_tools.h"


int main(int argc, char** argv){
  //reads files contents into memory
  //writes out a new file with its contents
  //ensure the files are the same byte for byte

  // Create a variable that will hold the address of where
  // our data is loaded into.
  unsigned char* data;

  // Load the data into that area and get back the number of
  // bytes read.
  size_t read_file_size = read_data("./shared/audio/backwards.wav", &data);

  // Prints the number of bytes read from the original file.
  printf("%lu bytes read with read_data().\n", read_file_size);

  // Copies the data into a different area and returns the file_size
  size_t written_file_size = write_data("./shared/audio/copied_backwards.wav", read_file_size, data);

  // Prints the number of bytes written to the copy.
  printf("%lu bytes written with write_data().\n", written_file_size);

  // Make sure the sizes match
  if (written_file_size != read_file_size) {
    printf("Error writing data. File sizes don't match.");
    free(data);
    return 1;
  }

  //
  unsigned char* written_data;
  size_t copy_file_size = read_data("./shared/audio/copied_backwards.wav", &written_data);

  // Make sure the sizes match again
  if (copy_file_size != read_file_size) {
    printf("Error reading copied data. File sizes don't match.");
    free(data);
    return 1;
  }

  // Check to see if the files are identical, byte for byte
  bool identical = true;
  for (size_t i = 0; i < read_file_size; i++) {
    if (data[i] != written_data[i]) {
    identical = false;
    break;
    }
  }

  if (identical) {
    printf("The files are identical, byte for byte. \n");
  } else {
    printf("The files are different. \n");
  }

  free(data);
  free(written_data);

  printf("\n");
  return 0;
}

Writing reversal.c


Now, let's compile those two files and run them:

In [ ]:
!gcc reversal.c file_tools.c -o reversal

> **Note:** You could replace ```gcc``` with ```clang``` here.  It doesn't matter which compiler you choose.

In [ ]:
!./reversal

1472176 bytes read with read_data().
1472176 bytes written with write_data().
The files are identical, byte for byte. 



# Thinking about it

Let's take a few moments and think about what is going on.  Answer the following questions before you proceed.

1.  We created an ```unsigned char``` pointer in ```main```.  What is the scope of that variable (which parts of the code can access it)?
2.  We didn't pass the value of the ```data``` variable to the ```read_data``` function.  Instead, we passed the address of the ```data``` variable.  Why?
3.  In our ```read_data``` function signature, we took a pointer to a pointer for the ```data``` variable.  Why?
4.  How many variables are named ```data``` in this simple program?  Are they the same variable?
5.  Why did we need to use indirection when storing the address returned from the ```malloc```?  More specifically, why did we need to write

 ```*data = malloc(file_size);```

 instead of

 ```data = malloc(file_size);```

 (notice a lack of '*' at the beginning)?
6.  Look at the ```write_data``` function signature.  It doesn't take a pointer to a pointer for the ```data``` variable.  Instead, it just takes a pointer.  Why?


## Put your answers here:

1. This variable will only be accessable within the main function (local) because it is defined within main.
2. *data* is just a pointer to a char (char\*) in main, and *read_data()* asks for a pointer to a pointer to a char (char\*\*) so we must take the address of *data* (&*data*) as the argument.
3. In our *read_data*, we use a pointer to a pointer because we do not yet know how much memory we need to allocate yet, until after we calculate the file size.
4. In *main()*, there is `unsigned char* data;`which is pointer to a char, which the address of this variable is passed into *read_data()* as an argument `read_data("./shared/audio/backwards.wav", &data);`, so this data `unsigned char** data` is a different variable.
The one in *main()* is a pointer to a char, while the one in *read_data()* is a pointer to a pointer to a char.
5. We don't want to change the memory address that is at data, we just want to allocate a spot in memory for data to point to.
6. The write() function doesn't take a pointer to a pointer because the size of the data is already known so we won't need to allocate for it within the function.



# Assignment Part 1

Now, complete the code for the ```write_data``` function.  You can write your code in the same box above where I left the function stub (the ```return 0``` in the ```write_data``` function).  You will need to look up and learn how to use the ```fwrite()``` function.

This function is given a filename, a number specifying the number of bytes to write, and a string of bytes.  Why must we pass the number of bytes as a parameter?  Why can't we simply find out the length from the string?  Well, C doesn't have strings.  What we have is a ```char*```, which is the address where the "string" starts.  We can't ask the address how long it is - it is just an address!

Modify the ```main``` function above so that it reads a file's contents into memory and then writes out a new file with the contents.  Ensure that the two files are **exactly** the same - byte for byte.  To do this, you may want to check the size of the files, or use the ```hexdump``` command.  This command will print out the contents of a file in hex.  For instance:

In [ ]:
%%writefile hello_world
Hello world!  I love computers.

Writing hello_world


In [ ]:
!hexdump -C hello_world

00000000  48 65 6c 6c 6f 20 77 6f  72 6c 64 21 20 20 49 20  |Hello world!  I |
00000010  6c 6f 76 65 20 63 6f 6d  70 75 74 65 72 73 2e 0a  |love computers..|
00000020


# Assignment Part 2

In the code above, we worked with text files.  To the computer though, text is just bytes.  So there is no reason we can't use the same techniques to work with a binary file.  We are going to work with .wav files for this project.  A .wav file is a type of uncompressed audio file.

Microphones take measurements of audio called **samples**.  For an audio file, it is not uncommon for their to be 44,100 samples *per second* - or even more!  These files include both metadata and samples.  Like most other file types, .wav files are very well structured.  For instance, we know the following:

- The first four bytes of the file will be the ASCII characters 'RIFF'.
- The next four bytes are the size of the file with 8 bytes subtracted.  This value is stored as a 32-bit integer.
- The third set of four bytes are the ASCII characters 'WAVE'.
- Bytes at the 22nd and 23rd position are for the number of channels, stored as a 16-bit integer.  If you have stereo sound, there will be two channels (a left and a right channel).  Mono sound will have only one channel.

Let's see if we can make sense of a sample file.  I've created a file called ```backwards.wav```.  Let's inspect the first 44 bytes of it:

In [ ]:
!file ./shared/audio/backwards.wav
!echo
!hexdump -Cn 44 ./shared/audio/backwards.wav

./shared/audio/backwards.wav: RIFF (little-endian) data, WAVE audio, Microsoft PCM, 16 bit, mono 44100 Hz

00000000  52 49 46 46 a8 76 16 00  57 41 56 45 66 6d 74 20  |RIFF.v..WAVEfmt |
00000010  10 00 00 00 01 00 01 00  44 ac 00 00 88 58 01 00  |........D....X..|
00000020  02 00 10 00 64 61 74 61  84 76 16 00              |....data.v..|
0000002c


I just output the first 44 bytes (the metadata) of the file; there are **way** more bytes in the file that make up the samples.  We can see the 'RIFF' and 'WAVE' parts already.  If we look closely at the output above, you will notice I ran the ```file``` command first.  This command tells us somethings about the file.  For instance, it shows that the file is mono (1 channel), and has a sample rate of 44100 Hz.  It isn't immediately obvious from the output, but these values are stored within the metadata of the file.  For instance, if we look at the bytes at location 22 and 23 we get "01 00".  On this system, this 16-bit (2-byte) value equals our decimal value of 1.  Since we know that bytes 22 and 23 are used to store the number of channels, we have just verified that our file is indeed mono.

> **Side note:** How do we get 1 from "01 00"?  Well, this system is what we call **little endian**.  This means that the least significant bit of a number comes first.  If we were writing our decimal numbers in a little endian way, we would write the number "42" as "24".  So really, the number we have stored for the number of channels is "0001".

Let's now look at the four bytes starting at location 24.  These bytes store the sample rate.  We should see 44100 here.  I see the bytes "44 ac 00 00".  Since this is a little endian system this is really "00 00 ac 44".  If you find an online hex converter and type in "00 00 ac 44", you will get 44100 as the converted value.  Very cool!

# Working with data

If we use the code we've already written, we can load the sample wave file into a byte array.  We don't need to make any changes to it!

Our ```read_data``` function simply pulls all the bytes off the disk and puts them into memory somewhere.  We can think of the area of memory as an array of bytes.  That's really all a ```char``` array is in C - ```char``` and ```byte``` are basically equivalent.  The difference is that we normally think of a ```char``` as something visible - but it doesn't have to be.  For instance, the new-line and tab characters, the system bell, the escape key, all of these are stored as a byte.  Just because we can't see them doesn't mean the computer doesn't need a way to store their values.

So, we have a big chunk of bytes.  We know that some of these bytes are single characters and some are values stored as multiple byte sequences.  We just need to interpret them the correct way.  Let's try to print out the values of the first four bytes of the file.  We know those should be the characters 'RIFF':

In [ ]:
%%writefile reversal.c

#include <stdio.h>
#include <string.h>
#include "file_tools.h"

int main(int argc, char** argv){
  unsigned char* data;

  size_t file_size = read_data("./shared/audio/backwards.wav", &data);

  printf("Size: %lu bytes.\n", file_size);
  printf("First four bytes: %c %c %c %c", data[0], data[1], data[2], data[3]);

  printf("\n");
  return 0;
}

Overwriting reversal.c


In [ ]:
!gcc reversal.c file_tools.c -o reversal
!./reversal

Size: 1472176 bytes.
First four bytes: R I F F


That looks correct!

But what if we had interpreted those bytes differently?  What if we wanted to see their hex values?  We can tell C to change how it interprets the bytes by changing the print specifier.  In the code above, we used the '%c' to print a character.  Let's modify it to use '%x' - that will print the hex value:

In [ ]:
%%writefile reversal.c

#include <stdio.h>
#include <string.h>
#include "file_tools.h"

int main(int argc, char** argv){
  unsigned char* data;

  size_t file_size = read_data("./shared/audio/backwards.wav", &data);

  printf("Size: %lu bytes.\n", file_size);
  printf("First four bytes: %x %x %x %x", data[0], data[1], data[2], data[3]);

  printf("\n");
  return 0;
}

Overwriting reversal.c


In [ ]:
!gcc reversal.c file_tools.c -o reversal
!./reversal

Size: 1472176 bytes.
First four bytes: 52 49 46 46


> **Note:** BE SURE YOU UNDERSTAND THIS PART!  We didn't change any of the values from the file, we just asked C to interpreted them differently!

Awesome!  That's easy enough.  But what if we wanted to tell C to look at a certain memory address as if it were a 4-byte integer?  For instance, how could we have C find the value for the sample rate?

The print specifier for an integer is '%d'.  Let's see what we get!

In [ ]:
%%writefile reversal.c

#include <stdio.h>
#include <string.h>
#include "file_tools.h"

int main(int argc, char** argv){
  unsigned char* data;

  size_t file_size = read_data("./shared/audio/backwards.wav", &data);

  printf("Size: %lu bytes.\n", file_size);
  printf("Sample rate: %d", data[24]);

  printf("\n");
  return 0;
}

Overwriting reversal.c


In [ ]:
!gcc reversal.c file_tools.c -o reversal
!./reversal

Size: 1472176 bytes.
Sample rate: 68


That's not 44100...  What gives?

The data type of our array is ```char*```.  This means that C should interpret each byte individually.  So, when we tell it to print element 24 as an integer, it grabs the one byte at location 24 and converts it to an integer.  That's not what we want.  We want it to take the four bytes starting at that location and interpret them as an integer.

But how do we do that?

It turns out to be fairly easy.  Since C allows us to cast a data type to another type, we can tell C to view the data at an address as another type of data.  We know the sample rate starts at location 24.  We know the address of that location can be found by using the '&' operator.  So, the address is ```&data[24]```.  Currently, C views the data at that address as a single char, since the data type is ```char*``` (an address of a ```char```).  But... we can cast that pointer to a different type of pointer, thus causing C to view the data the way we want:

In [ ]:
%%writefile reversal.c

#include <stdio.h>
#include <string.h>
#include "file_tools.h"

int main(int argc, char** argv){
  unsigned char* data;

  size_t file_size = read_data("./shared/audio/backwards.wav", &data);

  printf("Size: %lu bytes.\n", file_size);
  int value = *(int*)&data[24];
  printf("Sample rate: %d", value);

  printf("\n");
  return 0;
}

Overwriting reversal.c


In [ ]:
!gcc reversal.c file_tools.c -o reversal
!./reversal

Size: 1472176 bytes.
Sample rate: 44100


IT WORKS!

Let's explain the line a bit:

```C
int value
```

Create a new integer called value.

```C
&data[24]
```

Get the address of data element 24.

```C
(int*)&data[24]
````

Cast the address of data element 24 to an address of an integer.

```C
*(int*)&data[24]
```

Go into that address and get the data.  Since we've told C to view the address as an address of an integer, it will automatically take the four bytes.

# Ok.  Now what?

Well, now that we understand more about working with data in C, let's do something cool!

I want you to make a program that will get information about a wav file - then modify the file so that it will play backward.  We can't simply reverse all the bytes in the file though, as some are metadata, and others are samples that are more than one byte long.  What we need to do is

- copy the metadata
- reverse the *samples* (not the bytes)

Let's say that you have the following samples:

_________________________
| 00 01 | 02 03 | 04 05 |
_________________________

If we wanted this audio to play backward, we would need to rewrite it so that it looks like this:

_________________________
| 04 05 | 02 03 | 00 01 |
_________________________

The first 44 bytes of the file are the metadata.  You can copy the metadata without modifying it to the new file.  Then, you need to copy each sample from the old file to the new one, in reverse.  The wave file I have given you is very easy to work with; each sample is 16-bits (2-bytes), and there is only one channel.  The samples start at byte number 44.  Here is the original file:



In [ ]:
%%writefile reversal.c

#include <stdio.h>
#include "file_tools.h"


int main(int argc, char** argv){

  // Created a variable for the original file data
  unsigned char* data;

  // Load the original file data into that area and get back the number of
  // bytes read.
  size_t read_file_size = read_data("./shared/audio/backwards.wav", &data);

  // Allocate memory for the reversed file data
  unsigned char* reversed_data = malloc(read_file_size);

  // Copy the first 44 bytes of metadata unchanged
  for (size_t i = 0; i < 44; i++) {
        reversed_data[i] = data[i];
  }

  // Reverse the samples (2 bytes) by incrementing / decrementing by 2.
  // i is the index for the original file (increment).
  // j is the index for the reversed file (decrement).
  for (size_t i = 44, j = read_file_size - 2; i < read_file_size; i += 2, j -= 2) {
    reversed_data[j] = data[i]; // First byte
    reversed_data[j + 1] = data[i + 1]; // Second byte
  }

  // Write the reversed data to the new file
  write_data("reversed.wav", read_file_size, reversed_data);

  // Free allocated memory
  free(data);
  free(reversed_data);

  return 0;
}

Overwriting reversal.c


In [ ]:
!gcc reversal.c file_tools.c -o reversal
!./reversal

In [ ]:
!file ./shared/audio/backwards.wav
!echo
!hexdump -Cn 44 ./shared/audio/backwards.wav

!file reversed.wav
!echo
!hexdump -Cn 44 reversed.wav

./shared/audio/backwards.wav: RIFF (little-endian) data, WAVE audio, Microsoft PCM, 16 bit, mono 44100 Hz

00000000  52 49 46 46 a8 76 16 00  57 41 56 45 66 6d 74 20  |RIFF.v..WAVEfmt |
00000010  10 00 00 00 01 00 01 00  44 ac 00 00 88 58 01 00  |........D....X..|
00000020  02 00 10 00 64 61 74 61  84 76 16 00              |....data.v..|
0000002c
reversed.wav: RIFF (little-endian) data, WAVE audio, Microsoft PCM, 16 bit, mono 44100 Hz

00000000  52 49 46 46 a8 76 16 00  57 41 56 45 66 6d 74 20  |RIFF.v..WAVEfmt |
00000010  10 00 00 00 01 00 01 00  44 ac 00 00 88 58 01 00  |........D....X..|
00000020  02 00 10 00 64 61 74 61  84 76 16 00              |....data.v..|
0000002c


In [ ]:
display(Audio('reversed.wav', autoplay=True))

Make sure you output your file to a different name.  Use the code in the above cell to play your sound file after you reverse it.  If you did it correctly, you will be able to hear words (and you might recognize the voice).

# Grading

I will grade the submission on the following requirements.  For each requirement, you will recieve either 10, 5, or 0 points depending on whether you get excellent, meets expectations, or deficient.  For style guide, we will be using Google's C++ Style Guide.  See me if you have questions about it.  https://google.github.io/styleguide/cppguide.html

Be sure to separate your code into functions that each do **one important thing**.  For instance, your ```read_data()``` function should do just that - read data.  Same for your write function.  If we keep those functions pure - so that they do only what they say they do - we can use them more easily in other projects later.  So don't, for instance, put reversal code in your write function.  Similarly, functions that do a similar task should be in their own files.  You wouldn't put a function that reverses the bytes in the file_tools files.


| Requirement | Exceeds            | Meets              | Deficient        |
|-------------|--------------------|--------------------|------------------|
| Code is well commented according to the style guide given. | Two or fewer errors in documentation. | 2 to 5 errors in documentation.   | More than 5 errors in documentation. |
| Correctly outputs all of the required values. | Up to one value missing or incorrect.  | More than 1 value missing or incorrect. |
| All questions answered correctly above. | Two or fewer missing answers/incorrect answers. | More than two missing or incorrect answers.
| Sound file is reversed perfectly. | Two or fewer missing/incorrect bytes.  | More than 2 missing/incorrect bytes. |
| Code is properly separated into files. | Code is not separated into areas of concern properly. | Code is all in one file. |

You can download a copy of this Colab to your Google Drive, by clicking on the File menu and selecting "Save a Copy in Drive".  You don't need to upload any files, just copy the share link for this Colab and add that in the comments section on the Blackboard submission.
